In [4]:
import os, pandas, numpy as np
from amplpy import AMPL

In [27]:
# Initialize AMPL object
ampl = AMPL()

# Directory where the instance files are located
instance_dir = "instances"

# Ask user for the instance number to solve
instance = int(input("Enter the instance number to solve (1-21): "))
instance_file = os.path.join(instance_dir, f"inst{instance:02d}.dat")

# Read the first line of the instance file to get the value of n
with open(instance_file, 'r') as file:
    m = int(file.readline().strip())
    n = int(file.readline().strip())
    D = []
    l = np.array(file.readline().strip().split(), dtype=int)
    _ = file.readline()


    for i in range(n+1):
        d_split = file.readline().split()
        d_row = []
        for j in d_split:
            d_row.append(int(j))
        D.append(d_row)
    Dnp = np.array(D)
        

# Check if the matrix is symmetric
is_symmetric = np.array_equal(Dnp, Dnp.T)

# Load the AMPL model
if is_symmetric:
    ampl.read("model1_sb.mod")
else:
    ampl.read("model1.mod")

#Starting from D compute Upper and Lower bound for the problem:
round_trips = []
for i in range(n):
    round_trips.append(Dnp[i][n] + Dnp[n][i])
LB = max(round_trips)
#LLB = min(round_trips)

UB = 0
for i in range(n):
    UB += max(Dnp[i, :])

ampl.get_parameter("Q").set(max(l))
ampl.get_parameter("LB").set(LB)
ampl.get_parameter("UB").set(UB)

# Implementing hierarchical constarint type 1 (HC1)

# Partition the couriers based on their load capacity
courier_partitions = {}
for idx, capacity in enumerate(l):
    if capacity not in courier_partitions:
        courier_partitions[capacity] = []
    courier_partitions[capacity].append(idx + 1)

# Print the partitions
# for capacity, couriers in courier_partitions.items():
#     print(f"Couriers with capacity {capacity}: {couriers}")

# Given a courier with index k, find the courier with the same capacity and the largest index smaller than k
# for k in range(2, m+1):
#     # Get the capacity of the given courier
#     courier_capacity = l[k - 1]

#     # Find the couriers with the same capacity and smaller index
#     same_capacity_couriers = [idx for idx in courier_partitions[courier_capacity] if idx < k]

#     # Get the courier with the largest index among them
#     if same_capacity_couriers:
#         largest_index_courier = max(same_capacity_couriers)
#         print(f"The courier with the same capacity and the largest index smaller than {k} is: {largest_index_courier}")
#     else:
#         print(f"No courier with the same capacity and smaller index than {k} found.")

# for k in range(2, m+1):
#     courier_capacity = l[k - 1]
#     same_capacity_couriers = [idx for idx in courier_partitions[courier_capacity] if idx < k]
#     if same_capacity_couriers:
#         largest_index_courier = max(same_capacity_couriers)
#         for j in range(1, n+1):
#             ampl.eval(f"subject to HC1_{k}_{j}: sum {{i in 1..n+1}} x[i, {j}, {k}]  <= sum {{j_prime in 1..{j}-1}} sum {{i in 1..n+1}} x[i, j_prime, {largest_index_courier}];")

# Hierarchical constraint on first item delivered
# for k1 in range(1, m):
#     for k2 in range(k1+1, m+1):
#         if l[k1-1] == l[k2-1]:
#             for i in range(1, n+1):
#                 for j in range(i+1, n+1):
#                     ampl.eval(f"subject to HC1_{k1}_{k2}_{i}_{j}: x[n+1,{j},{k1}] + x[n+1,{i},{k2}] <= 1;")


# Load the data for the current instance
instance_file = os.path.join(instance_dir, f"inst{instance:02d}_transformed.dat")
ampl.readData(instance_file)

In [3]:
for variable in ampl.get_variables():
    print(f"Variable: {variable[1].name()}")
    if variable[1].name() == 'maxDist':
        print("Lower bound:", variable[1].lb())
        print("Upper bound:", variable[1].ub())
    print()

Variable: maxDist
Lower bound: 8.0
Upper bound: 40.0

Variable: u

Variable: x



### For second model (light encoding)

In [ ]:
# Solve the model
#ampl.solve(solver="cbc", cbc_options="seconds=300 timing=2")
ampl.solve(solver="highs", highs_options="timelim=10 timing=2")

# Get the solution for variables (e.g., x, u, maxDist)
A = ampl.getVariable("A").getValues()
B = ampl.getVariable("B").getValues()

#u = ampl.getVariable("u").getValues()
maxDist = ampl.getVariable("maxDist").value()

# Print the results (or process them as needed)
print(f"\nResults for instance inst{instance:02d}.dat:")
print(f"A: {A}")
print(f"B: {B}")
#print(f"u: {u}")
print(f"maxDist: {maxDist}")
print("-" * 40)

### For first model (larger encoding)

In [28]:
import re

# Solve the model
ampl.setOption('randseed', 42) 

#ampl.solve(solver="cbc", cbc_options="seconds=300 timing=2 randomCbcSeed=42 randomSeed=42")
#ampl.solve(solver="highs", highs_options="timelim=300 timing=2")
ampl.setOption("solver", "gurobi")
ampl.setOption("gurobi_options", "timelim=300 timing=2 seed=42")
#ampl.solve(solver="gurobi", gurobi_options="timelim=300 timing=2 seed=42")
#ampl.solve(solver="xpress", xpress_options="timelim=300 timing=2")
#ampl.solve(solver="cplex", cplex_options="timelim=300 timing=2")
#ampl.solve(solver="copt", copt_options="timelim=300 timing=2")
#ampl.solve(solver="mosek", mosek_options="timelim=300 timing=2 seed=42")

solver_output = ampl.get_output("solve;")
solver_time_match = re.search(r"Solver time = ([\d.]+)s", solver_output)
solver_time = float(solver_time_match.group(1)) if solver_time_match else None
print(f"Solver time: {solver_time} seconds")

# Get the solution for the variables
x = ampl.getVariable("x").getValues()
u = ampl.getVariable("u").getValues()
#distTraveled = ampl.getVariable("distTraveled").getValues()
maxDist = ampl.getVariable("maxDist").value()

xnp = x.to_pandas().values.reshape((n+1, n+1, m))
# Print the path of each courier
for k in range(1, m + 1):
    path = []
    current_node = n + 1
    while True:
        next_node = None
        for i in range(1, n + 1):
            if xnp[current_node-1, i-1, k-1] > 0.5:  # Check if the courier travels from current_node to node i
                next_node = i
                break
        if next_node is None:
            break
        path.append(next_node)
        current_node = next_node
    print(f"Courier {k} path: {'--->'.join(map(str, path))}")

# Print the results (or process them as needed)
print(f"\nResults for instance inst{instance:02d}.dat:")
print(f"x: {x}")
print(f"u: {u}")
#print(f"distTraveled: {distTraveled}")
print(f"maxDist: {maxDist}")
print("-" * 40)

Solver time: 0.226647 seconds
Courier 1 path: 1--->3--->4
Courier 2 path: 2--->5--->6

Results for instance inst01.dat:
x:    index0       index1       index2    |    x.val    
     1              1              1       |      0      
     1              1              2       |      0      
     1              2              1       |      0      
     1              2              2       |      0      
     1              3              1       |      1      
     1              3              2       |      0      
     1              4              1       |      0      
     1              4              2       |      0      
     1              5              1       |      0      
     1              5              2       |      0      
     1              6              1       |      0      
     1              6              2       |      0      
     1              7              1       |      0      
     1              7              2       |      0      
     2     